# Estimation of failure probability due to wave overtopping

In this example, we will demonstrate the application of various reliability methods (`numerical_bisection`, `latin_hypercube` and `cobyla_reliability`) to estimate the probability of failure of a levee due to wave overtopping. 

### Define model

First, we import the necessary packages:

In [1]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod, StandardNormal
import numpy as np

We consider the following limit state function:

$Z = q_{crit} - q_{o}$

where: <br>

* $q_{crit}$ is the critical overtopping discharge (m3/s/m) <br>
* $q_{o}$ is the occuring wave overtopping discharge (m3/s/m) <br>

The wave overtopping discharche $q_{o}$ is derived in this example using simplified equations from [*Technical Report Wave Run-up and Wave Overtopping at Dikes*, Technical Advisory Committee on Flood Defence, The Netherlands, Delft, May 2002.]. The wave overtopping depends on:

* local water level $h$ (m) <br>
* significant wave height $H_{m0}$ (m) <br>
* spectral wave period $T_{m-1,0}$ (s) <br>
* wave direction with respect to North <br>
* dike normal with respect to North <br>
* crest height $y_{crest}$ (m) <br>

In [2]:
def overtopping(h, hm0, tm10, wave_direction, dike_normal, y_crest):

    f_shallow = 0.6778
    f_b = 4.30
    f_n = 2.30
    gamma_f = 1.0
    gamma_b = 1.0
    tan_alpha = 1/3
    g = 9.81

    angle = np.abs(wave_direction-dike_normal)
    if angle<=180.0:
        beta = angle
    else:
        beta = 360.0-angle

    if beta>110.0:
        hm0 = 0.0
        tm10 = 0.0
    
    if beta>80.0 and beta<=110.0:
        hm0 = hm0*(110.0-beta)/30.0
        tm10 = tm10*np.sqrt((110.0-beta)/30.0)

    gamma_beta0 = 1.0-0.0033*min([beta,80.0])

    l0 = g/(2*np.pi)*tm10**2
    s0 = hm0/l0

    eta_0 = tan_alpha/np.sqrt(s0)
    eta_shallow = max([7.0,eta_0])
    term = np.sqrt(g*hm0**3)

    q_b = (0.067/np.sqrt(tan_alpha))*gamma_b*eta_0*np.exp(-f_b*((y_crest-h)/hm0)*(1/(gamma_beta0*gamma_f*gamma_b*eta_0)))
    q_n = 0.2*np.exp(-f_n*((y_crest-h)/hm0)*(1/(gamma_beta0*gamma_f)))
    q_shallow = (10**(-f_shallow))*np.exp(-(y_crest-h)/(hm0*gamma_beta0*gamma_f*(0.33+0.022*eta_shallow)))

    q_0 = 0.0
    if eta_0<=5.0:
        q_0 = term*min([q_b, q_n])

    elif eta_0>=7.0:
        q_0 = term*q_shallow

    else:
        if q_n>0.0 and q_shallow>0.0:
            q_0 = term*max([q_n, np.exp(np.log(q_n)+(np.log(q_shallow)-np.log(q_n)))*(eta_0-5.0)/(7.0-5.0)])
        else:
            q_0= 0.0

    if hm0<0.05:
        q_0 = 0.0

    if h>y_crest:
        q_0 = np.nan

    return q_0

The limit state function is:

In [3]:
def z_func_overtopping(h, hm0, tm10, wave_direction, dike_normal, y_crest, q_crit):

    if h>y_crest:
        return y_crest - h
    else:  
        return q_crit - overtopping(h, hm0, tm10, wave_direction, dike_normal, y_crest)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [4]:
project = ReliabilityProject()
project.model = z_func_overtopping

We assume the following distributions for the parameters present in the limit state function:

In [5]:
project.variables["h"].distribution = DistributionType.log_normal
project.variables["h"].mean = 1.5
project.variables["h"].deviation = 0.05

project.variables["hm0"].distribution = DistributionType.log_normal
project.variables["hm0"].mean = 1.5
project.variables["hm0"].deviation = 0.25

project.variables["tm10"].distribution = DistributionType.log_normal
project.variables["tm10"].mean = 3
project.variables["tm10"].deviation = 0.5

project.variables["wave_direction"].distribution = DistributionType.deterministic
project.variables["wave_direction"].mean = 0.0

project.variables["dike_normal"].distribution = DistributionType.deterministic
project.variables["dike_normal"].mean = 0.0

project.variables["y_crest"].distribution = DistributionType.deterministic
project.variables["y_crest"].mean = 6.0

project.variables["q_crit"].distribution = DistributionType.log_normal
project.variables["q_crit"].mean = 0.001
project.variables["q_crit"].deviation = 0.01

### Perform reliability calculations with Numerical bisection

We start with the reliability method `numerical_bisection`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [6]:
project.settings.reliability_method = ReliabilityMethod.numerical_bisection
project.settings.minimum_iterations = 5
project.settings.maximum_iterations = 6
project.settings.epsilon_beta = 0.1

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Beta = 1.2741637321172252
Probability of failure = 0.10130269778906441
h: alpha = -0.04532582838055149, x = 1.5020553608678822
hm0: alpha = -0.3366611410569041, x = 1.5884680172871342
tm10: alpha = -0.62787186521049, x = 3.3781755946626975
wave_direction: alpha = 0.0, x = 0.0
dike_normal: alpha = 0.0, x = 0.0
y_crest: alpha = 0.0, x = 6.0
q_crit: alpha = 0.7002726370928543, x = 1.4634234350708521e-05
Not converged (convergence = 0.3327398400248571 > 0.05)
Model runs = 213583


### Perform reliability calculations with Latin hypercube

We now conduct the reliability analysis using the `latin_hypercube`.

In [7]:
project.settings.reliability_method = ReliabilityMethod.latin_hypercube
project.settings.minimum_samples = 25000

project.run()

read_results(project.design_point)

Beta = 1.297672201302545
Probability of failure = 0.09720000000000073
h: alpha = -0.04443744963518474, x = 1.5020509919293492
hm0: alpha = -0.34821841244887497, x = 1.5945037673445286
tm10: alpha = -0.6226671689601575, x = 3.382655511452117
wave_direction: alpha = 0.0, x = 0.0
dike_normal: alpha = 0.0, x = 0.0
y_crest: alpha = 0.0, x = 6.0
q_crit: alpha = 0.6993245648485746, x = 1.4163112089984726e-05
Not converged (convergence = 0.019274922405676582 > 0.05)
Model runs = 25001


### Perform reliability calculations with Cobyla algorithm

We now conduct the reliability analysis using the `cobyla_reliability` method.

In [8]:
project.settings.reliability_method = ReliabilityMethod.cobyla_reliability
project.settings.maximum_iterations = 150
project.settings.epsilon_beta = 0.01

project.run()

read_results(project.design_point)

Beta = 1.3373058910507976
Probability of failure = 0.0905614044211556
h: alpha = 0.028059918355148657, x = 1.4972938594493919
hm0: alpha = -0.14048878517878133, x = 1.526326899749656
tm10: alpha = -0.2974723854478765, x = 3.160597529369696
wave_direction: alpha = 0.0, x = 0.0
dike_normal: alpha = 0.0, x = 0.0
y_crest: alpha = 0.0, x = 6.0
q_crit: alpha = 0.9439204003075907, x = 6.608753380327907e-06
Not converged (convergence = nan > 0.05)
Model runs = 151
